In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import KFold

In [9]:

# Fungsi untuk membangun model
# def build_model():
#     model = Sequential([
#         Flatten(input_shape=(28, 28)),
#         Dense(128, activation='relu'),
#         Dense(60, activation='softmax')
#     ])
#     return model

# Fungsi
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [10]:
# Membuat dataset menggunakan tf.keras.preprocessing.image_dataset_from_directory
mnist = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=32,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)

Found 240 files belonging to 60 classes.


In [12]:
# Preprocessing data
def preprocess_data(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Preprocess data train
mnist = mnist.map(preprocess_data)

In [13]:
# Pemilihan kombinasi batch size dan jumlah epoch yang akan dievaluasi
batch_sizes = [30, 32]
num_epochs = [100, 1000]

In [14]:
# Inisialisasi k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

In [15]:
# Mengeksekusi eksperimen untuk setiap kombinasi
results = {}
for batch_size in batch_sizes:
    for num_epoch in num_epochs:
        print(f"Training with batch size {batch_size} and {num_epoch} epochs...")
        val_accuracies = []

        # Mengambil indeks dari dataset
        indices = np.arange(len(mnist))

        for train_index, val_index in kf.split(indices):
            # Mengambil data pelatihan dan validasi menggunakan indeks
            # error out of range
            train_data = mnist.unbatch().filter(lambda x, i: tf.reduce_any(tf.equal(i, train_index))).batch(batch_size)
            val_data = mnist.unbatch().filter(lambda x, i: tf.reduce_any(tf.equal(i, val_index))).batch(batch_size)

            model = build_resnet50_model()
            model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=[SparseCategoricalAccuracy()])

            # Melatih model dengan kombinasi yang dipilih
            history = model.fit(train_data, epochs=num_epoch, verbose=0)

            # Evaluasi model pada data validasi
            # _, val_accuracy = model.evaluate(val_data, verbose=0)
            # val_accuracies.append(val_accuracy)

        # Menyimpan hasil evaluasi
        # results[(batch_size, num_epoch)] = np.mean(val_accuracies)
        results[(batch_size, num_epoch)] = history.history

Training with batch size 30 and 100 epochs...


2024-03-25 07:48:09.452840: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-25 07:48:09.751215: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-25 07:48:10.066230: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-25 07:48:10.295308: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-25 07:48:10.544356: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-25 07:48:10.768710: W tensorflow/core/framework/local_rendezvous.cc:404]

KeyboardInterrupt: 

In [ ]:
# Analisis hasil
best_accuracy = 0
best_combination = None
for combination, val_accuracy in results.items():
    val_accuracy = history['val_sparse_categorical_accuracy'][-1]
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_combination = combination

    print(f"Batch size: {combination[0]}, Epochs: {combination[1]}")
    print("Average Validation Accuracy:", val_accuracy)

print(f"Best combination: batch size {best_combination[0]}, {best_combination[1]} epochs. Validation accuracy: {best_accuracy}")